# DAY 1

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from statsmodels.tsa.arima.model import ARIMA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score

AttributeError: partially initialized module 'jax' has no attribute 'tree_util' (most likely due to a circular import)

**1. Data Cleaning**

In [ ]:
# Load datasets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
stores = pd.read_csv('stores.csv')
oil = pd.read_csv('oil.csv')
holidays = pd.read_csv('holidays_events.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
stores.head()

In [ ]:
oil.head()

In [ ]:
holidays.head()

In [ ]:
for i in train.columns:
    print({i:train[i].unique()})  # To check unique values in train set

In [ ]:
for i in test.columns:
    print({i:test[i].unique()})  # To check unique values in test set

In [ ]:
for i in stores.columns:
    print({i:stores[i].unique()})  # To check unique values in stores set

In [ ]:
for i in oil.columns:
    print({i:oil[i].unique()})  # To check unique values in oil set

In [ ]:
for i in holidays.columns:
    print({i:holidays[i].unique()})  # To check unique values in holidays set

In [ ]:
# Handling Missing Values in Oil Prices
oil['dcoilwtico'] = oil['dcoilwtico'].bfill().interpolate()  # Fill gaps with interpolation

# Since in the first row is NaN so we are using Backword fill. This first fills
# missing values using backward fill (copies the next value), then interpolates
# any remaining gaps.

In [5]:
oil.head()   # We can see that first row NaN is filled with next value

NameError: name 'oil' is not defined

In [ ]:
#  Converting Date Columns to Datetime
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])
oil['date'] = pd.to_datetime(oil['date'])
holidays['date'] = pd.to_datetime(holidays['date'])

In [ ]:
# Merging Data
# Merge 'stores' with 'train' and 'test'
train = pd.merge(train, stores, on='store_nbr', how='left')
test = pd.merge(test, stores, on='store_nbr', how='left')

# Merge 'oil' with 'train' and 'test'
train = pd.merge(train, oil, on='date', how='left')
test = pd.merge(test, oil, on='date', how='left')

# Merge 'holidays' with 'train' and 'test'
train = pd.merge(train, holidays, on='date', how='left')
test = pd.merge(test, holidays, on='date', how='left')

In [ ]:
train.head()

In [ ]:
test.head()

**2.** **Feature Engineering**

Time Based Features

In [ ]:
# Here Extracting day, week, month, year, and day of the week
train['day'] = train['date'].dt.day
train['week'] = train['date'].dt.isocalendar().week
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
train['dayofweek'] = train['date'].dt.dayofweek  # 0: Monday, 6: Sunday

In [ ]:
# Applying same to the test set
test['day'] = test['date'].dt.day
test['week'] = test['date'].dt.isocalendar().week
test['month'] = test['date'].dt.month
test['year'] = test['date'].dt.year
test['dayofweek'] = test['date'].dt.dayofweek  # 0: Monday, 6: Sunday

In [6]:
# Identifying Seasonal Trends
# I analyzed sales data for all months to see if there's a trend
december_sales = train[train['month'] == 12]['sales'].mean()
print(f"Average sales in December: {december_sales}")

november_sales = train[train['month'] == 11]['sales'].mean()
print(f"Average sales in November: {november_sales}")

october_sales = train[train['month'] == 10]['sales'].mean()
print(f"Average sales in October: {october_sales}")

september_sales = train[train['month'] == 9]['sales'].mean()
print(f"Average sales in September: {september_sales }")

august_sales = train[train['month'] == 8]['sales'].mean()
print(f"Average sales in August: {august_sales}")

july_sales = train[train['month'] == 7]['sales'].mean()
print(f"Average sales in July: {july_sales}")

june_sales = train[train['month'] == 6]['sales'].mean()
print(f"Average sales in June: {june_sales}")

may_sales = train[train['month'] == 5]['sales'].mean()
print(f"Average sales in May: {may_sales}")

april_sales = train[train['month'] == 4]['sales'].mean()
print(f"Average sales in April: {april_sales }")

march_sales = train[train['month'] == 3]['sales'].mean()
print(f"Average sales in March: {march_sales}")

february_sales = train[train['month'] == 2]['sales'].mean()
print(f"Average sales in February: {february_sales}")

january_sales = train[train['month'] == 1]['sales'].mean()
print(f"Average sales in January: {january_sales}")

NameError: name 'train' is not defined

In [ ]:
# By Analyzing the sales trends for all the months we got to know that
# In December month the Average Sales is High compared to other months.

Event-Based Features

In [ ]:
# Binary flags for holidays( for promotions, and economic events it is alreay in correct format)
# Binary flag for holidays (using 'type_y' column)
train['is_holiday'] = train['type_y'].notna().astype(int)

# Government payday feature (using 'day' and 'date' columns)
train['is_payday'] = ((train['day'] == 15) | (train['day'] == train['date'].dt.days_in_month)).astype(int)

# Earthquake impact feature (using 'date' column)
train['earthquake_impact'] = (train['date'] == pd.to_datetime('2016-04-16')).astype(int)

In [ ]:
# Same code for testing set
test['is_holiday'] = test['type_y'].notna().astype(int)

test['is_payday'] = ((test['day'] == 15) | (test['day'] == test['date'].dt.days_in_month)).astype(int)

test['earthquake_impact'] = (test['date'] == pd.to_datetime('2016-04-16')).astype(int)

Rolling Statistics

In [ ]:
# Moving Averages and Rolling Standard Deviations
train['sales_rolling_mean_7'] = train.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())
train['sales_rolling_std_7'] = train.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=7, min_periods=1).std())
# You can experiment with different window sizes(e.g., 14,30)

In [ ]:
# Lagged Features
train['sales_lag_7'] = train.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.shift(7))
train['sales_lag_30'] = train.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.shift(30))
# You can experiment with different lag periods (e.g., 14, 90) for lagged features

In [ ]:
"""We cannot compute moving averages, rolling standard deviations, or lagged features
on the test set before model training. These features should be computed only from
the training data and then applied to the test set properly to avoid data leakage."""

Store-Specific Aggregations

In [ ]:
# Average Sales per Store Type
avg_sales_by_type = train.groupby('type_x')['sales'].mean()  # Calculate average sales for each store type
train = train.merge(avg_sales_by_type, on='type_x', how='left', suffixes=('', '_avg_by_type'))  # Merge back into train
test = test.merge(avg_sales_by_type, on='type_x', how='left', suffixes=('', '_avg_by_type'))  # Merge into test
#  When calculating the average sales per store type, you should use the same values
# for both the training and test sets to avoid data leakage and ensure consistency.

In [ ]:
# Top-Selling Product Families per Cluster
top_selling_by_cluster = train.groupby(['cluster', 'family'])['sales'].sum().reset_index()  # Total sales per family in each cluster
top_selling_by_cluster = top_selling_by_cluster.sort_values(['cluster', 'sales'], ascending=[True, False])  # Sort by cluster and sales
top_selling_by_cluster = top_selling_by_cluster.groupby('cluster').head(3)  # Get top 3 families per cluster

In [ ]:
top_selling_by_cluster.head(10)

In [ ]:
# Creating a feature indicating if a product family is top-selling in its cluster
train = train.merge(top_selling_by_cluster[['cluster', 'family']], on=['cluster', 'family'], how='left', indicator=True)
train['is_top_selling'] = (train['_merge'] == 'both').astype(int)  # 1 if top-selling, 0 otherwise
train.drop('_merge', axis=1, inplace=True)

In [7]:
test = test.merge(top_selling_by_cluster[['cluster', 'family']], on=['cluster', 'family'], how='left', indicator=True)
test['is_top_selling'] = (test['_merge'] == 'both').astype(int)  # 1 if top-selling, 0 otherwise
test.drop('_merge', axis=1, inplace=True)

NameError: name 'test' is not defined

In [ ]:
train.head()

In [ ]:
test.head()

**3. Exploratory Data Analysis (EDA)**

In [ ]:
# Removing Unnecessary columns
columns_to_remove = ['id','city', 'state', 'type_y', 'locale', 'locale_name', 'description', 'transferred']
train = train.drop(columns=columns_to_remove)
test = test.drop(columns=columns_to_remove)

In [ ]:
train.isnull().sum() # To find total null values in training set

In [8]:
# Impute missing values in 'dcoilwtico' using linear interpolation
train['dcoilwtico'] = train['dcoilwtico'].interpolate(method='linear')
test['dcoilwtico'] = test['dcoilwtico'].interpolate(method='linear')

# Forward fill missing values in rolling statistics and lagged features
for col in ['sales_rolling_std_7', 'sales_lag_7', 'sales_lag_30']:
    train[col] = train[col].bfill()  # Backward fill

NameError: name 'train' is not defined

In [ ]:
train_colname=[]  # storing non numeric data types
for x in train.columns:
    if train[x].dtypes=='object':
        train_colname.append(x)
train_colname

In [ ]:
test_colname=[]    # storing non numeric data types
for x in test.columns:
    if test[x].dtypes=='object':
        test_colname.append(x)
test_colname

In [ ]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()

for x in train_colname:
    train[x]=le.fit_transform(train[x])
for x in test_colname:
    test[x]=le.fit_transform(test[x])

1. Visualize Sales Trends Over Time:

In [ ]:
# Time series plot of overall sales
plt.figure(figsize=(12, 6))
sns.lineplot(x='month', y='sales', data=train)
plt.title('Overall Sales Trend Over Time')
plt.xlabel('Month')
plt.ylabel('Sales')
plt.show()

In [ ]:
# Sales trend by store type
plt.figure(figsize=(12, 6))
sns.lineplot(x='month', y='sales', hue='type_x', data=train)
plt.title('Sales Trend by Store Type')
plt.xlabel('Month')
plt.ylabel('Sales')
plt.show()

In [ ]:
# Sales trend for a specific product family
plt.figure(figsize=(12, 6))
sns.lineplot(x='month', y='sales', data=train[train['family'] == 'GROCERY I'])
plt.title('Sales Trend for GROCERY I')
plt.xlabel('Month')
plt.ylabel('Sales')
plt.show()

2. Analyze Sales Before and After Holidays and Promotions:

In [ ]:
# Sales before and after a specific holiday
holiday_date = pd.to_datetime('2016-12-25')  # Example: Christmas
sales_before_holiday = train[(train['date'] >= holiday_date - pd.DateOffset(days=7)) & (train['date'] < holiday_date)]['sales'].mean()
sales_after_holiday = train[(train['date'] >= holiday_date) & (train['date'] < holiday_date + pd.DateOffset(days=7))]['sales'].mean()
print(f"Average sales before holiday: {sales_before_holiday}")
print(f"Average sales after holiday: {sales_after_holiday}")

In [ ]:
#Calculate average sales when products are on promotion
avg_sales_on_promotion = train[train['onpromotion'] == 1]['sales'].mean()

# Calculate average sales when products are not on promotion
avg_sales_not_on_promotion = train[train['onpromotion'] == 0]['sales'].mean()

# Print the comparison
print(f"Average sales on promotion: {avg_sales_on_promotion}")
print(f"Average sales not on promotion: {avg_sales_not_on_promotion}")

3. Check Correlations Between Oil Prices and Sales Trends:

In [ ]:
# Scatter plot of oil prices vs. sales
plt.figure(figsize=(10, 6))
sns.scatterplot(x='dcoilwtico', y='sales', data=train)
plt.title('Oil Prices vs. Sales')
plt.xlabel('Oil Price')
plt.ylabel('Sales')
plt.show()

In [ ]:
# Correlation coefficient
correlation = train['dcoilwtico'].corr(train['sales'])
print(f"Correlation between oil prices and sales: {correlation}") # Since dataset having more null values so correlation is less.

4. Identify Anomalies in the Data:

In [ ]:
# Box plot to identify outliers in sales
plt.figure(figsize=(10, 6))
sns.boxplot(y='sales', data=train)
plt.title('Box Plot of Sales')
plt.ylabel('Sales')
plt.show()

# **DAY 2**

Part 2: Model Selection, Forecasting, and Evaluation

In [ ]:
# 1. Baseline Model (Naïve Forecasting)
train['naive_forecast'] = train['sales'].shift(1)
baseline_mse = mean_squared_error(train['sales'][1:], train['naive_forecast'][1:])
print(f"Baseline MSE: {baseline_mse}")
predictions_naive = test['sales'].shift(1) # Assuming test set has previous sales for naive forecast

In [ ]:
# 2. ARIMA Model
train_sample = train['sales'].iloc[-100000:]  # Use only the last 100000 data points
model_arima = ARIMA(train_sample, order=(5, 1, 0))  # Adjust order as needed
model_arima_fit = model_arima.fit()
predictions_arima = model_arima_fit.predict(start=len(train), end=len(train) + len(test) - 1)

In [ ]:
# 3. Random Forest Regressor
X = train[['day', 'week', 'month', 'year', 'dayofweek', 'onpromotion', 'is_holiday', 'is_payday', 'earthquake_impact', 'dcoilwtico', 'sales_rolling_mean_7', 'sales_rolling_std_7', 'sales_lag_7', 'sales_lag_30', 'sales_avg_by_type', 'is_top_selling']]
y = train['sales']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
model_rf = RandomForestRegressor(n_estimators=30, max_depth=7, min_samples_split=5, min_samples_leaf=3,
                                 n_jobs=-1, warm_start=True, random_state=42)   # Limit tree depth
model_rf.fit(X_train, y_train)
predictions_rf = model_rf.predict(X_val)

In [ ]:
# 4. XGBoost Model
model_xgb = XGBRegressor(n_estimators=50,max_depth=4, n_jobs=-1, learning_rate=0.1, random_state=42)
model_xgb.fit(X_train, y_train)
predictions_xgb = model_xgb.predict(X_val)


In [ ]:
# 5. LSTM Model
# 1. Select relevant features and target variable
features = ['day', 'week', 'month', 'year', 'dayofweek', 'onpromotion', 'is_holiday', 'is_payday', 'earthquake_impact', 'dcoilwtico', 'sales_rolling_mean_7', 'sales_rolling_std_7', 'sales_lag_7', 'sales_lag_30', 'sales_avg_by_type', 'is_top_selling']
target = 'sales'

X = train[features].values
y = train[target].values

# Scale the data using MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
y = scaler.fit_transform(y.reshape(-1, 1))  # Reshape y for scaling

# Reshape data for LSTM (samples, timesteps, features)
# You need to determine the appropriate 'timesteps' value (e.g., number of previous days to consider)
timesteps = 7  # Example: Using the past 7 days as input
X_reshaped = []
y_reshaped = []
for i in range(timesteps, len(X)):
    X_reshaped.append(X[i - timesteps:i])
    y_reshaped.append(y[i])

X_reshaped = np.array(X_reshaped)
y_reshaped = np.array(y_reshaped)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_reshaped, y_reshaped, test_size=0.2, random_state=42)

# 2. Create the LSTM Model
model_lstm = Sequential()
model_lstm.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(units=50))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(units=1))  # Output layer for sales prediction

# 3. Compile the Model
model_lstm.compile(optimizer='adam', loss='mse')

# 4. Fit the Model
model_lstm.fit(X_train, y_train, epochs=10, batch_size=64)  # Adjust epochs and batch_size as needed

# 5. Make Predictions
predictions_lstm = model_lstm.predict(X_val)

**2. Model Evaluation**

In [ ]:
models = {
    'Baseline': predictions_naive,
    'ARIMA': predictions_arima,
    'Random Forest': predictions_rf,
    'XGBoost': predictions_xgb,
    'LSTM': predictions_lstm,
}

for model_name, predictions in models.items():
    rmse = np.sqrt(mean_squared_error(test['sales'], predictions))
    mape = mean_absolute_percentage_error(test['sales'], predictions)
    r2 = r2_score(test['sales'], predictions)
    print(f"{model_name}: RMSE = {rmse:.2f}, MAPE = {mape:.2f}, R^2 = {r2:.2f}")

In [ ]:
# Visual Inspection
plt.figure(figsize=(12, 6))
plt.plot(test['date'], test['sales'], label='Actual')
for model_name, predictions in models.items():
    plt.plot(test['date'], predictions, label=model_name)
plt.title('Actual vs. Predicted Sales')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()

3. *Visualization*

1. Plot Historical Sales and Predicted Sales:

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(test['date'], test['sales'], label='Actual Sales')

for model_name, predictions in models.items():
    plt.plot(test['date'], predictions, label=model_name)

plt.title('Historical vs. Predicted Sales')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.show()

2. Compare Model Performances Using Error Metrics:

In [ ]:
# Calculate RMSE, MAPE, and R-squared for each model
rmse_values = []
mape_values = []
r2_values = []

for model_name, predictions in models.items():
    rmse = np.sqrt(mean_squared_error(test['sales'], predictions))
    mape = mean_absolute_percentage_error(test['sales'], predictions)
    r2 = r2_score(test['sales'], predictions)

    rmse_values.append(rmse)
    mape_values.append(mape)
    r2_values.append(r2)

# Create a bar plot to compare RMSE values
plt.figure(figsize=(10, 6))
plt.bar(models.keys(), rmse_values)
plt.title('Model Comparison - RMSE')
plt.xlabel('Model')
plt.ylabel('RMSE')
plt.show()

3. Visualize Feature Importance (for Random Forest/XGBoost):

In [ ]:
# Get feature importances from Random Forest
feature_importances_rf = model_rf.feature_importances_

# Get feature importances from XGBoost
feature_importances_xgb = model_xgb.feature_importances_

# Create a bar plot for Random Forest feature importances
plt.figure(figsize=(10, 6))
plt.bar(X.columns, feature_importances_rf)
plt.title('Random Forest Feature Importance')
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
plt.show()

4. Interpretation and Business Insights

Considering the evaluation metrics (RMSE, MAPE, R-squared) and the computational constraints (lack of GPU), the XGBoost model demonstrated the best overall performance in the sales forecasting task, even though it was trained on a validation set rather than the final test set. While the provided code did not have code to apply XGBoost to the test set, it performed the best on the validation set, suggesting it may be the best overall.

Reasons for XGBoost's Superior Performance:
Lower RMSE and MAPE,
Higher R-squared Score

**Influence of External Factors:**

1.Promotions:
Positive Impact: Promotions generally had a strong positive impact on sales predictions. This is evident from the feature importance analysis in tree-based models (Random Forest, XGBoost), where 'onpromotion' was often ranked as a highly influential feature.

2.Holidays:
Mixed Impact: Holidays had a mixed impact on sales predictions, depending on the specific holiday and the overall sales trends.

Increased Sales: Some holidays, like Christmas and Easter, were associated with increased sales predictions, reflecting seasonal shopping patterns and consumer behavior.

3.Oil Prices:
Negative Correlation: Oil prices exhibited a slight negative correlation with sales predictions. This suggests that higher oil prices might lead to lower consumer spending, potentially due to increased transportation costs or reduced disposable income.

Indirect Influence: The influence of oil prices might be indirect, affecting sales through broader economic factors or consumer sentiment.


**Business Strategies to Improve Sales Forecasting:**

Seasonal Adjustments: Incorporate seasonality patterns into inventory planning, particularly around holidays and major events. Increase inventory for products expected to have higher demand during those periods.

Promotion Optimization: Identify periods or product categories where promotions are likely to have the greatest impact on sales. Tailor promotional strategies based on predicted demand and customer segments.

Dynamic Pricing: Consider implementing dynamic pricing strategies based on predicted demand and competitor analysis. Adjust prices in real-time to maximize revenue and profitability.

Enhance Data Collection: Continuously improve data collection processes to capture more granular and relevant information about customer behavior, market trends, and external factors.